### 本篇代码用于建立对照模型——SVM

In [ ]:

from sklearn.metrics import roc_auc_score
from keras.utils import np_utils,plot_model, multi_gpu_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from sklearn.model_selection import StratifiedShuffleSplit #sklearn是python自带的机器学习库
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix

import pandas as pd
import scipy
from scipy import sparse

In [ ]:
df = pd.read_pickle("Pickles/processed_data.pkl")
#df = pd.read_csv("Data/lemmatized_text.csv")

In [ ]:
df.head()

#### 把项目变量变成n-1个哑变量

In [ ]:
mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('items')),columns=mlb.classes_,),sort=False,how="left")

In [21]:
df.head()

,GICS Sector,GICS Sub Industry,cik,doc_name,release_date,text,ticker,txt_link,rm_week,rm_month,rm_qtr,rm_year,vix,price_change,signal,processed_text,text_len,sentiment,Item 1.01,Item 1.02,Item 2.01,Item 2.02,Item 2.03,Item 2.04,Item 2.05,Item 2.06,Item 3.01,Item 3.02,Item 3.03,Item 4.02,Item 5.01,Item 5.02,Item 5.03,Item 5.04,Item 5.07,Item 5.08,Item 7.01,Item 8.01,Item 9.01
0,Information Technology,Information Technology,789019.0,0001193125-19-022553.txt,2019-01-30 16:03:36,0001193125-19-022553.txt : 20190130 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...,-3.80,0.61,2.12,20.47,17.66,-1.82,down,"[accession, number, conformed, submission, typ...",980,0.066667,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Information Technology,Information Technology,789019.0,0001193125-18-337951.txt,2018-11-29 15:29:57,0001193125-18-337951.txt : 20181129 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...,6.77,0.11,-0.01,23.76,18.65,-0.95,stay,"[accession, number, conformed, submission, typ...",279,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,Information Technology,Information Technology,789019.0,0001193125-18-306365.txt,2018-10-24 16:03:06,0001193125-18-306365.txt : 20181024 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...,-0.91,-1.60,1.60,37.85,25.23,4.71,up,"[accession, number, conformed, submission, typ...",838,0.000000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Information Technology,Information Technology,789019.0,0001193125-18-277602.txt,2018-09-19 16:15:58,0001193125-18-277602.txt : 20180919 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...,1.02,2.87,9.60,42.83,11.75,1.35,up,"[accession, number, conformed, submission, typ...",474,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,Information Technology,Information Technology,789019.0,0001193125-18-221458.txt,2018-07-19 16:02:35,0001193125-18-221458.txt : 20180719 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...,0.66,5.09,8.96,38.96,12.87,5.21,up,"[accession, number, conformed, submission, typ...",1332,0.333333,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


#### 定义一个函数将文本连接起来方便后面向量化，为SVM建模做准备 （预处理时文本被tokenize）

In [ ]:

def combine_text(text):
    combined_text = []
    combined_text = ' '.join(str(x) for x in text)
    return combined_text

In [ ]:
df['processed_con_text'] = df['processed_text'].apply(combine_text)

#### 把其他类别变量变成哑变量，把label和数据分开

In [ ]:
# 把label和数据分开，所有类别变量都变成哑变量
cols = ['GICS Sector','vix','rm_week','rm_month', 'rm_qtr', 'rm_year','sentiment']
cols.extend(list(mlb.classes_))
X = df[cols]
docs = df['processed_text']

y = df['signal']

# Get Dummies

#docs = tokenize_and_pad(docs)
X = pd.get_dummies(columns = ['GICS Sector'],prefix="sector",data=X)
#y = pd.get_dummies(columns=['signal'],data=y)

#aux_shape = len(X.columns)

检查是否正确

In [25]:
len(y)

3426

In [24]:
len(X)

3426

In [23]:
len(docs)

3426

In [26]:
X.head()

,vix,rm_week,rm_month,rm_qtr,rm_year,sentiment,Item 1.01,Item 1.02,Item 2.01,Item 2.02,Item 2.03,Item 2.04,Item 2.05,Item 2.06,Item 3.01,Item 3.02,Item 3.03,Item 4.02,Item 5.01,Item 5.02,Item 5.03,Item 5.04,Item 5.07,Item 5.08,Item 7.01,Item 8.01,Item 9.01,sector_Communication Services,sector_Consumer Discretionary,sector_Consumer Staples,sector_Energy,sector_Financials,sector_Health Care,sector_Industrials,sector_Information Technology,sector_Materials,sector_Real Estate,sector_Utilities
0,17.66,-3.80,0.61,2.12,20.47,0.066667,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
1,18.65,6.77,0.11,-0.01,23.76,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,25.23,-0.91,-1.60,1.60,37.85,0.000000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
3,11.75,1.02,2.87,9.60,42.83,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
4,12.87,0.66,5.09,8.96,38.96,0.333333,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0


#### 将连续变量标准化

In [ ]:

cont_features = ['vix','rm_week','rm_month', 'rm_qtr', 'rm_year']

x_scaler = StandardScaler()
X[cont_features] = x_scaler.fit_transform(X[cont_features])



In [73]:
X.head()

,vix,rm_week,rm_month,rm_qtr,rm_year,sentiment,Item 1.01,Item 1.02,Item 2.01,Item 2.02,Item 2.03,Item 2.04,Item 2.05,Item 2.06,Item 3.01,Item 3.02,Item 3.03,Item 4.02,Item 5.01,Item 5.02,Item 5.03,Item 5.04,Item 5.07,Item 5.08,Item 7.01,Item 8.01,Item 9.01,sector_Communication Services,sector_Consumer Discretionary,sector_Consumer Staples,sector_Energy,sector_Financials,sector_Health Care,sector_Industrials,sector_Information Technology,sector_Materials,sector_Real Estate,sector_Utilities
0,0.709172,-1.521789,-0.050842,-0.025137,0.386731,0.066667,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
1,0.943801,2.566365,-0.146495,-0.255412,0.517772,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,2.503252,-0.404025,-0.473627,-0.081354,1.078978,0.000000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
3,-0.691490,0.342440,0.381508,0.783529,1.277332,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
4,-0.426052,0.203203,0.806205,0.714338,1.123190,0.333333,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0


#### 将数据划分为training,testing

In [ ]:

X_train, X_test, y_train, y_test, docs_train, docs_test = train_test_split(X, y,docs,
                                                    stratify=y, 
                                                    test_size=0.2,
                                                    random_state = 20)
#X_validation, X_test, y_validation, y_test, docs_validation, docs_test = train_test_split(X_test_va, y_test_va,docs_test_va,
 #                                                   stratify=y_test_va, 
  #                                                  test_size=0.5,
   #                                                 random_state = 20)

In [70]:
y.head()

0    down
1    stay
2      up
3      up
4      up
Name: signal, dtype: object

#### oversample数据调整训练集中的样本不均衡问题

In [ ]:
def oversample(X,docs,y):
    # Get number of rows with imbalanced class
    target = y.sum().idxmax()#得到y中数量最多的对象，up
    n = y[target].sum()#计数
    # identify imbalanced targets
    imbalanced = y.drop(target,axis=1) #扔掉up，保留另外两列
    #For each target, create a dataframe of randomly sampled rows, append to list
    #随机从另外两列中抽取一些数据进行复制
    append_list =  [y.loc[y[col]==1].sample(n=n-y[col].sum(),replace=True,random_state=20) for col in imbalanced.columns]
    append_list.append(y)
    #复制时他们的index也复制了，所以可以用来匹配X
    y = pd.concat(append_list,axis=0)
    # match y indexes on other inputs
    X = X.loc[y.index] #注意这里是根据两个表的索引来匹配的，一旦索引除了问题就会出错
    docs = docs.loc[y.index]
    assert (y.index.all() == X.index.all() == docs.index.all())
    return X,docs,y

In [52]:
#调整样本代表性
X_train_svm, docs_train_svm, y_train_svm = oversample(X_train, docs_train, y_train)

In [ ]:
# Save data

X_train_svm.to_pickle("Pickles/X_train_svm.pkl")
X_test_svm.to_pickle("Pickles/X_test_svm.pkl")
#X_validation.to_pickle("Pickles/X_validation.pkl")

y_train_svm.to_pickle("Pickles/y_train_svm.pkl")
y_test_svm.to_pickle("Pickles/y_test_svm.pkl") 

docs_train_svm.to_pickle("Pickles/docs_train_svm")
docs_test_svm.to_pickle("Pickles/docs_test_svm")


#### 导入模型进行训练

先定义一个函数来计算多分类问题的auc

In [ ]:
def auc_svm(test,pred,average='macro'):
    lb=LabelBinarizer()
    lb.fit(test)
    test=lb.transform(test)
    pred=lb.transform(pred)
    return roc_auc_score(test,pred,average=average)

先用文本数据进行训练和测试

In [7]:
basicvectorizer = CountVectorizer(ngram_range=(1,1))
svm_vec_train = basicvectorizer.fit_transform(docs_train_svm)
print(svm_vec_train.shape)

(5172, 105327)


In [8]:
svm_vec_test = basicvectorizer.transform(docs_test_svm)
print(svm_vec_test.shape)

(686, 105327)


In [37]:
svm_model = svm.LinearSVC(C=1,max_iter=10000)
#svm_model_trained = svm_model.fit([svm_vec_train,X_train_svm], y_train_svm)
svm_model_trained = svm_model.fit(svm_vec_train, y_train_svm)

C:\Users\xueli\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


其实从这里就可以看出这个文本代表方法的缺陷——模型的迭代设置到了10000（默认值的10倍），仍然无法收敛

In [38]:
predictions = svm_model_trained.predict(svm_vec_test)

In [102]:
print (classification_report(y_test_svm, predictions))


              precision    recall  f1-score   support

        down       0.37      0.45      0.40       142
        stay       0.41      0.41      0.41       112
          up       0.73      0.68      0.70       432

   micro avg       0.59      0.59      0.59       686
   macro avg       0.50      0.51      0.51       686
weighted avg       0.60      0.59      0.59       686



采用macro的方法，因为auc的函数也是采用的这个指标。上面这个报告里的precision就是每一种分类里面的accuracy，他们的平均就是整体的accuracy.

In [40]:
auc_svm(y_test_svm, predictions)

0.6313629016021873

用非文本数据进行训练和测试

In [32]:
svm_model = svm.LinearSVC(C=1,max_iter=10000)
#svm_model_trained = svm_model.fit([svm_vec_train,X_train_svm], y_train_svm)
svm_model_trained = svm_model.fit(X_train_svm, y_train_svm)

In [33]:
predictions = svm_model_trained.predict(X_test_svm)

In [34]:
print (classification_report(y_test_svm, predictions))

              precision    recall  f1-score   support

        down       0.33      0.20      0.25       142
        stay       0.27      0.54      0.36       112
          up       0.70      0.61      0.65       432

   micro avg       0.51      0.51      0.51       686
   macro avg       0.43      0.45      0.42       686
weighted avg       0.55      0.51      0.52       686



In [36]:
auc_svm(y_test_svm, predictions)

0.5859298974808187

用整体数据进行训练和测试

In [ ]:
train_all = sparse.hstack((svm_vec_train, X_train_svm)).tocsr()#将文本向量和普通变量合在一起

In [ ]:
test_all = sparse.hstack((svm_vec_test, X_test_svm)).tocsr()

In [ ]:
svm_model = svm.LinearSVC(C=1,max_iter=10000)
#svm_model_trained = svm_model.fit([svm_vec_train,X_train_svm], y_train_svm)
svm_model_trained = svm_model.fit(train_all, y_train_svm)

In [ ]:
predictions = svm_model_trained.predict(test_all)

In [21]:
print (classification_report(y_test_svm, predictions))


              precision    recall  f1-score   support

        down       0.39      0.44      0.42       142
        stay       0.38      0.40      0.39       112
          up       0.73      0.69      0.71       432

   micro avg       0.59      0.59      0.59       686
   macro avg       0.50      0.51      0.51       686
weighted avg       0.60      0.59      0.60       686



In [31]:
auc_svm(y_test_svm,predictions)

0.6331499930845349